In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.utils import to_categorical
import numpy as np
 
#Load Data with pandas, and parse the first column into datetime
# train=pd.read_csv('train.csv', parse_dates = ['Dates'])
# test=pd.read_csv('test.csv', parse_dates = ['Dates'])
data=pd.read_csv('data/combined_weather_crime_lunar_df_two.csv')
data.head()

/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

,Unnamed: 0,DATE,CCN,STATION,NAME,DEWP,GUST,MAX,MIN,PRCP,SLP,SNDP,TEMP,WDSP,Year,Month,Date,Day,Illum
0,0,1/1/16,103,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",30.8,0.0,57.0,39.9,0.0,1020.6,0.0,44.0,8.0,2016,1,1,6,0.60
1,1,1/2/16,93,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",23.4,14.0,48.0,32.0,0.0,1020.2,0.0,39.5,6.4,2016,1,2,7,0.50
2,2,1/3/16,100,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",28.0,24.1,54.0,32.0,0.0,1013.9,0.0,40.9,5.1,2016,1,3,1,0.41
3,3,1/4/16,77,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",18.2,28.9,54.0,30.0,0.0,1017.8,0.0,35.6,12.5,2016,1,4,2,0.32
4,4,1/5/16,92,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",3.0,29.9,37.9,16.0,0.0,1034.3,0.0,24.2,12.7,2016,1,5,3,0.23


In [2]:
data.drop(data.columns[[0]], axis=1, inplace=True)
data.head()

,DATE,CCN,STATION,NAME,DEWP,GUST,MAX,MIN,PRCP,SLP,SNDP,TEMP,WDSP,Year,Month,Date,Day,Illum
0,1/1/16,103,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",30.8,0.0,57.0,39.9,0.0,1020.6,0.0,44.0,8.0,2016,1,1,6,0.60
1,1/2/16,93,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",23.4,14.0,48.0,32.0,0.0,1020.2,0.0,39.5,6.4,2016,1,2,7,0.50
2,1/3/16,100,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",28.0,24.1,54.0,32.0,0.0,1013.9,0.0,40.9,5.1,2016,1,3,1,0.41
3,1/4/16,77,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",18.2,28.9,54.0,30.0,0.0,1017.8,0.0,35.6,12.5,2016,1,4,2,0.32
4,1/5/16,92,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",3.0,29.9,37.9,16.0,0.0,1034.3,0.0,24.2,12.7,2016,1,5,3,0.23


In [3]:
data.rename(columns={"CCN": "CRIME-COUNT"}, inplace=True)

In [4]:
data['bins'] = np.nan

In [5]:
data.head()

,DATE,CRIME-COUNT,STATION,NAME,DEWP,GUST,MAX,MIN,PRCP,SLP,SNDP,TEMP,WDSP,Year,Month,Date,Day,Illum,bins
0,1/1/16,103,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",30.8,0.0,57.0,39.9,0.0,1020.6,0.0,44.0,8.0,2016,1,1,6,0.60,NaN
1,1/2/16,93,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",23.4,14.0,48.0,32.0,0.0,1020.2,0.0,39.5,6.4,2016,1,2,7,0.50,NaN
2,1/3/16,100,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",28.0,24.1,54.0,32.0,0.0,1013.9,0.0,40.9,5.1,2016,1,3,1,0.41,NaN
3,1/4/16,77,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",18.2,28.9,54.0,30.0,0.0,1017.8,0.0,35.6,12.5,2016,1,4,2,0.32,NaN
4,1/5/16,92,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",3.0,29.9,37.9,16.0,0.0,1034.3,0.0,24.2,12.7,2016,1,5,3,0.23,NaN


In [6]:
# data.loc[data['CRIME-COUNT']<=75, ['bins']] = 'VeryLow'


In [7]:
# data.loc[(data['CRIME-COUNT']>=76) & (data['CRIME-COUNT']<=90), ['bins']] = 'Low'

In [8]:
# data.loc[(data['CRIME-COUNT']>=91) & (data['CRIME-COUNT']<=105), ['bins']] = 'Medium'

In [9]:
# data.loc[(data['CRIME-COUNT']>=106) & (data['CRIME-COUNT']<=120), ['bins']] = 'High'

In [10]:
# data.loc[(data['CRIME-COUNT']>=121, ['bins'])] = 'VeryHigh'

In [11]:
data.loc[data['CRIME-COUNT']<=90, ['bins']] = 'Low'

In [12]:
data.loc[(data['CRIME-COUNT']>=91) & (data['CRIME-COUNT']<=120), ['bins']] = 'Medium'

In [13]:
data.loc[data['CRIME-COUNT']>=121, ['bins']] = 'VeryLow'

In [14]:
data['bins'].value_counts()

Medium     655
Low        584
VeryLow     80
Name: bins, dtype: int64

In [15]:
data['CRIME-COUNT'].describe()

count    1319.000000
mean       93.359363
std        17.072853
min        28.000000
25%        82.000000
50%        93.000000
75%       104.000000
max       146.000000
Name: CRIME-COUNT, dtype: float64

In [16]:
data.isnull().any()

DATE           False
CRIME-COUNT    False
STATION        False
NAME           False
DEWP           False
GUST           False
MAX            False
MIN            False
PRCP           False
SLP            False
SNDP           False
TEMP           False
WDSP           False
Year           False
Month          False
Date           False
Day            False
Illum          False
bins           False
dtype: bool

In [17]:
data.head()

,DATE,CRIME-COUNT,STATION,NAME,DEWP,GUST,MAX,MIN,PRCP,SLP,SNDP,TEMP,WDSP,Year,Month,Date,Day,Illum,bins
0,1/1/16,103,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",30.8,0.0,57.0,39.9,0.0,1020.6,0.0,44.0,8.0,2016,1,1,6,0.60,Medium
1,1/2/16,93,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",23.4,14.0,48.0,32.0,0.0,1020.2,0.0,39.5,6.4,2016,1,2,7,0.50,Medium
2,1/3/16,100,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",28.0,24.1,54.0,32.0,0.0,1013.9,0.0,40.9,5.1,2016,1,3,1,0.41,Medium
3,1/4/16,77,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",18.2,28.9,54.0,30.0,0.0,1017.8,0.0,35.6,12.5,2016,1,4,2,0.32,Low
4,1/5/16,92,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",3.0,29.9,37.9,16.0,0.0,1034.3,0.0,24.2,12.7,2016,1,5,3,0.23,Medium


In [18]:
X = data.drop(["bins", "DATE", "STATION", "NAME", "CRIME-COUNT", "Year"], axis=1)
y = data["bins"]

In [19]:
data.head()

,DATE,CRIME-COUNT,STATION,NAME,DEWP,GUST,MAX,MIN,PRCP,SLP,SNDP,TEMP,WDSP,Year,Month,Date,Day,Illum,bins
0,1/1/16,103,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",30.8,0.0,57.0,39.9,0.0,1020.6,0.0,44.0,8.0,2016,1,1,6,0.60,Medium
1,1/2/16,93,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",23.4,14.0,48.0,32.0,0.0,1020.2,0.0,39.5,6.4,2016,1,2,7,0.50,Medium
2,1/3/16,100,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",28.0,24.1,54.0,32.0,0.0,1013.9,0.0,40.9,5.1,2016,1,3,1,0.41,Medium
3,1/4/16,77,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",18.2,28.9,54.0,30.0,0.0,1017.8,0.0,35.6,12.5,2016,1,4,2,0.32,Low
4,1/5/16,92,72405013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",3.0,29.9,37.9,16.0,0.0,1034.3,0.0,24.2,12.7,2016,1,5,3,0.23,Medium


In [20]:
# X['TMAX'] = X['TMAX'].astype(float)
# X['TMIN'] = X['TMIN'].astype(float)
# X['Month'] = X['Month'].astype(float)
# X['Date'] = X['Date'].astype(float)
# X['Day'] = X['Illum'].astype(float)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [22]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [24]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [26]:
model = Sequential()
model.add(Dense(units=300, activation='relu', input_dim=13))
model.add(Dropout(.2))
# model.add(Dense(units=400, activation='sigmoid'))
# model.add(Dense(units=100, activation='tanh'))
# model.add(Dropout(.2))
model.add(Dense(units=200, activation='relu'))
# model.add(Dropout(.2))
model.add(Dense(units=100, activation='tanh'))
# model.add(Dropout(.2))
model.add(Dense(units=3, activation='softmax'))

W0817 17:03:09.518426 4538688960 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [27]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               4200      
_________________________________________________________________
dropout (Dropout)            (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               60200     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 84,803
Trainable params: 84,803
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=500,
    shuffle=True,
    verbose=2
)

Epoch 1/500
989/989 - 0s - loss: 0.8811 - acc: 0.5541
Epoch 2/500
989/989 - 0s - loss: 0.8205 - acc: 0.6178
Epoch 3/500
989/989 - 0s - loss: 0.8030 - acc: 0.6249
Epoch 4/500
989/989 - 0s - loss: 0.7819 - acc: 0.6400
Epoch 5/500
989/989 - 0s - loss: 0.7910 - acc: 0.6168
Epoch 6/500
989/989 - 0s - loss: 0.7780 - acc: 0.6350
Epoch 7/500
989/989 - 0s - loss: 0.7635 - acc: 0.6481
Epoch 8/500
989/989 - 0s - loss: 0.7676 - acc: 0.6471
Epoch 9/500
989/989 - 0s - loss: 0.7713 - acc: 0.6390
Epoch 10/500
989/989 - 0s - loss: 0.7617 - acc: 0.6461
Epoch 11/500
989/989 - 0s - loss: 0.7498 - acc: 0.6542
Epoch 12/500
989/989 - 0s - loss: 0.7551 - acc: 0.6522
Epoch 13/500
989/989 - 0s - loss: 0.7831 - acc: 0.6198
Epoch 14/500


In [ ]:
# X_train_scaled

In [ ]:
# y_train_categorical.shape

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [ ]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")